In [1]:
from nmm import Qobj,spost,spre,cumulant,OverdampedBath
from numpy import random
import jax.numpy as jnp

In [2]:
A=random.rand(30,30)
B=random.rand(30,30)
C=random.rand(30,30)

In [3]:
A=Qobj(jnp.array(A))
B=Qobj(jnp.array(B))
C=Qobj(jnp.array(C))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
def test(A,B):
    m=spre(A)*spost(B)-0.5*spre(A*B) -0.5*spost(A*B)
    return m.expm()

In [5]:
%timeit test(A,B)

161 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
import jax

In [7]:
f=jax.jit(test)

In [8]:
%timeit f(A,B)

156 ms ± 8.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
A=Qobj(jnp.array([[1,0],[0,-1]]))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
A.eigenstates()

(Array([-1.,  1.], dtype=float32),
 Array([[0., 1.],
        [1., 0.]], dtype=float32))

In [4]:
evals, all_state = A.eigenstates()


In [26]:
import jax.numpy as jnp
from jax import jit

@jit
def jump_operators(Q, evals, evecs):
    N = evecs.shape[1]

    num_terms = N * (N - 1) // 2 * 2 + 1  # emission + absorption + dephasing
    collapse_list = jnp.zeros((Q.shape[0], Q.shape[1], num_terms), dtype=Q.dtype)
    ws = jnp.zeros(num_terms)

    idx = 0
    for j in range(N):
        for k in range(j + 1, N):
            Deltajk = evals[k] - evals[j]

            state_j = evecs[:, j:j+1]  # as column
            state_k = evecs[:, k:k+1]

            # emission term
            C1 = state_j * (state_j.dag() * Q * state_k) * state_k.dag()
            ws = ws.at[idx].set(Deltajk)
            collapse_list = collapse_list.at[:, :, idx].set(C1.data)
            idx += 1

            # absorption term
            C2 = state_k * (state_k.dag()* Q * state_j) * state_j.dag()
            ws = ws.at[idx].set(-Deltajk)
            collapse_list = collapse_list.at[:, :, idx].set(C2.data)
            idx += 1

    # Dephasing term
    Cdeph = Q - jnp.sum(collapse_list[:, :, :idx], axis=2)
    ws = ws.at[idx].set(0.0)
    collapse_list = collapse_list.at[:, :, idx].set(Cdeph)

    # Now group by rounded frequencies
    ws_rounded = jnp.round(ws, 12)

    unique_ws = jnp.unique(ws_rounded)
    dict_keys = unique_ws
    dict_values = jnp.zeros((Q.shape[0], Q.shape[1], unique_ws.shape[0]), dtype=Q.dtype)

    for u_idx in range(unique_ws.shape[0]):
        mask = (ws_rounded == unique_ws[u_idx])
        # sum collapse_list[:,:,mask] over axis=2
        summed = jnp.sum(collapse_list[:, :, mask], axis=2)
        dict_values = dict_values.at[:, :, u_idx].set(summed)

    return dict_keys, dict_values


In [27]:
all_state

Array([[0., 1.],
       [1., 0.]], dtype=float32)

In [28]:
jump_operators(A,evals,Qobj(all_state))

ValueError: Non-hashable static arguments are not supported, as this can lead to unexpected cache-misses. Static argument (index 0) of type <class 'nmm.utils.utils.Qobj'> for function __sub__ is non-hashable.

In [12]:
d=test(A,B)